[Previous Section](./Encryption.102.ipynb)

---

# Encryption 3 - RSA

This introduces the concept of public and private keys... Vast topic indeed!

Ronald **Ri**ivest, Adi **S**hamir, Leonard **A**dleman.

## Summary
- [Pre-requisites](#Pre-requisites)
- [In short](#in-short)
- [In details](#in-details)
  - Implementing utilities
  - P, Q, E, and D numbers
  - RSA utilities
- [For real, on one character](#For-real,-on-one-character)
- [For real, on a message](#For-real,-on-a-message)
- [Dependencies](#Dependencies)

### Pre-requisites
To understand what we are going to present here, you need to be somewhat familiar with the following concepts:  
- [One-way functions](https://en.wikipedia.org/wiki/One-way_function)
- Prime numbers
- [Big Integer](https://docs.oracle.com/javase/8/docs/api/?java/math/BigInteger.html)s
  - We are going to have to deal with **_very big_** integer numbers, going way beyond the Java `Integer.MAX_VALUE`, or even the `Long.MAX_VALUE`... The `java.math.BigInteger` class has been created for this kind of context, and if we consider the methods it implements, it sounds like it could have been implemented for this very kind of encryption context...

## In short
The idea here is to avoid the several message round trips we've seen before, with several layers of encoding.  
To do so, the concept of public and private key will be introduced.  

A message will be encoded with one key, and decoded with another one.

If - let's say - Bernard wants to send a message to Alice, he will use - to encrypt his message - a _**public key**_, published by Alice, in some kind of phone book.  
In order to decrypt Bernard's message, Alice will use her _**private key**_ - kept secret.  

A first question would be   
"_why can't Bernard's message be decrypted with the key used to encrypt it?_"  
A first answer would be  
"_Because this encryption is using - on purpose - one-way functions._"  

A function is an "operation" that transforms `Number A` into `Number B`. **One-way functions** are functions that do **not** let you deduct the value of `Number A`, if you only have `Number B`.

_Example_:  
The `square` function is _not_ a one-way function. For `square`, the function `f(x)` is defined as 
$$f(x) = x^2$$
If you're told that `f(x) = 9`, you can definitely deduct that `x = 3`. You use the `square root` function, which is the reciprocal function of `square`.

Now, let's take the `modulo` function, considered as a one-way function. This one has several definitions, a good one could be:  
`modulo(x, y)` (also noted `x modulo y`) is the remainder of the division of `x` by `y`.  
`13 modulo 5` is equal to `3`.  
If we consider here `5` as the encryption key, then the character `13` would be encryted as `3`.  
But how to decrypt it without a doubt?  
`13 modulo 5` is equal to `3`, but so are `93 modulo 5`, `618 modulo 5`, ... etc.  
To decrypt a message encoded with such a one-way function, you would need to do a big number of tests to reach an approximate solution.  
That is the context. This approach is not 100% uncrackable, but it requires such an amount of work to crack it that it can be considered as safe.

As we will see, the _private key_ depends on the value of the _public key_, obviously. And you would see how this dependency is coded.  
This document will only illustrate the way it works. Deeper details can be found in other documents.

## In details
First, we are going to implement to utility methods, that will be used all over the place.

In [1]:
import java.text.NumberFormat;
import java.util.stream.Collectors;

public class PrimeNumbers {

    public enum OutputOption {
        NONE, HTML, MARKDOWN, LATEX
    }

    private final static boolean VERBOSE = false; // Change at will

    private static void addToMap(Map<Integer, Integer> primeMap, int n) {
        primeMap.merge(n, 1, Integer::sum);
    }
    
    public static Map<Integer, Integer> primeFactors(int n) {
        Map<Integer, Integer> primeMap = new HashMap<>();

        // Print the number of 2s that divides n
        while (n % 2 == 0) {
            if (VERBOSE) {
                System.out.print(2 + " ");
            }
            addToMap(primeMap, 2);
            n /= 2;
        }

        // n must be odd at this point.  So we can skip one element (Note i = i + 2)
        for (int i = 3; i <= Math.sqrt(n); i += 2) {
            // While i divides n, print i and divide n
            while (n % i == 0) {
                if (VERBOSE) {
                    System.out.print(i + " ");
                }
                addToMap(primeMap, i);
                n /= i;
            }
        }

        // This condition is to handle the case when n is a prime number greater than 2
        if (n > 2) {
            if (VERBOSE) {
                System.out.print(n);
            }
            addToMap(primeMap, n);
        }
        if (VERBOSE) {
            System.out.println();
        }
        return primeMap;
    }

    public static String spitMapOut(int value, Map<Integer, Integer> primeMap) {
        return spitMapOut(value, primeMap, OutputOption.NONE);
    }
    
    public static String spitMapOut(int value, Map<Integer, Integer> primeMap, OutputOption outputOption) {
        final String collected = primeMap.keySet()
                .stream().map(prime -> {
                    String output;
                    switch (outputOption) {
                        case MARKDOWN:
                        case HTML:
                            output = String.format("(%d<sup>%d</sup>)", prime, primeMap.get(prime));
                            break;
                        case LATEX:
                            output = String.format("(%d^{%d})", prime, primeMap.get(prime));
                            break;
                        case NONE:
                        default:
                            output = String.format("(%d^%d)", prime, primeMap.get(prime));
                    }
                    return output;
                })
                .collect(Collectors.joining((outputOption == OutputOption.MARKDOWN || outputOption == OutputOption.HTML) ?
                        " &times; " :
                        (outputOption == OutputOption.LATEX ? " \\times " : " x ")));
        String finalResult;
        switch (outputOption) {
            case MARKDOWN:
                finalResult = String.format("`%s = %s`", NumberFormat.getInstance().format(value) ,collected);
                break;
            case HTML:
                finalResult = String.format("<code>%s = %s</code>", NumberFormat.getInstance().format(value) ,collected);
                break;
            case LATEX:
                finalResult = String.format("$ %s = %s $", NumberFormat.getInstance().format(value) ,collected);
                break;
            case NONE:
            default:
                finalResult = String.format("%s = %s", NumberFormat.getInstance().format(value) ,collected);
        }
        return finalResult;
    }
    
    public static int pgcd(int n1, int n2) {
        while (n1 != n2) {
            if (n1 > n2) {
                n1 -= n2;
            } else {
                n2 -= n1;
            }
        }
        return n2;
    }

    public static boolean isPrime(long num) {
        boolean isPrime = false;
        int count = 0;
        // Check for divisibility from 2 up to i/2
        for (int j = 2; j <= num / 2; j++) {
            if (num % j == 0) {
                count++;  // Increment if 'num' is divisible by 'j'
                break;    // Exit loop if a divisor is found. We don't need more than one.
            }
        }
        // If the count is 0, 'i' is prime
        if (count == 0) {
            isPrime = true;
        }
        return isPrime;
    }
}
// Examples
List<Long> list = List.of(499L, 1789L, 437L);
list.forEach(n -> System.out.printf("%d is%s prime.\n", n, PrimeNumbers.isPrime(n) ? "" : " not"));

499 is prime.
1789 is prime.
437 is not prime.


We are going to use several numbers here, named `P`, `Q`, `E`, and `D`.  
`E` will be the **public key** (along with `N`, which is `P x Q`).  
`D` will be the **private key** (along with `N` too).  
`P` and `Q` will be used to bridge those two keys together, we will see how, what to publish, what to keep secret.  
Let's start with `P`, `Q`, and `E`.

In [2]:
long aliceP = 127;
long aliceQ = 499;
long aliceE = 23_801;

The value of `E` strongly depends on the values of `P`, `Q`, and `D`.
We will see how later.  
First we define again some utility methods.

In [3]:
public class RSAUtils {
    
    public static boolean checkPQE(long p, long q, long e) {
        long D = findD(p, q, e);
        long phi = (p - 1) * (q - 1);
        return ((e * D) % phi) == 1;
    }

    public static boolean checkPhiDCoprime(long p, long q, long e) {
        long D = findD(p, q, e);
        long phi = (p - 1) * (q - 1);

        long pgcdPhiD = PrimeNumbers.pgcd((int)D, (int)phi);
        if (pgcdPhiD != 1) {
            System.out.printf("PGCD D, \u03D5 (%d, %d) : %d => %s\n", D, phi, pgcdPhiD, (pgcdPhiD == 1) ? "good" :
                    String.format("not good, \u03D5 => %s, D => %s", 
                                  PrimeNumbers.spitMapOut((int)phi, 
                                  PrimeNumbers.primeFactors((int)phi)), 
                                  PrimeNumbers.spitMapOut((int)D, PrimeNumbers.primeFactors((int)D))));
        }
        return (pgcdPhiD == 1);
    }

    /**
     * Calculate the Private Key
     * TODO Might need tuning...
     *
     * See https://www.geeksforgeeks.org/how-to-solve-rsa-algorithm-problems/#
     *
     * @param p
     * @param q
     * @param e
     * @return the private key for p, q, e
     */
    public static long findD(long p, long q, long e) {
        long phi = (p - 1) * (q - 1);
        // To solve => (e * d) % ((p - 1) * (q - 1)) = 1
        //          => (e * d) % phi = 1
        // pgcd(phi, d) = 1
        boolean found = false;
        long D = 1;
        while (!found) { // TODO there must be a better way... This one can loop forever.
            long x = (e * D) % phi;
            if (x == 1) {
                found = true;
            } else {
                D++;
            }
        }
        long d = D;

        return d;
    }

    /**
     * A Test...
     * Find a list of possible private keys for the context (p, q, e).
     * Stops when the value Integer.MAX_VALUE is reached for the key.
     * (it could be more, .ike Long.MAX_VALUE)
     */
    public static List<Long> findDList(long p, long q, long e) {
        List<Long> dList = new ArrayList<>();

        long phi = (p - 1) * (q - 1);
        // To solve => (e * d) % ((p - 1) * (q - 1)) = 1
        //          => (e * d) % phi = 1
        // pgcd(phi, d) = 1
        long D = 1;
        while (true) {
            long x = (e * D) % phi;
            if (x == 1) {
                dList.add(D);
            }
            D++;
            // if (D == Long.MAX_VALUE) {
            if (D == Integer.MAX_VALUE) {
                // Enough!... Exit loop.
                break;
            }
        }
        return dList;
    }

    public static int encodeWithPublicKey(long pkE, long pkN, int toEncode) {
        // Produce (toEncode ^ pkE) % pkN
        int encoded = (int)BigInteger.valueOf(toEncode).modPow(BigInteger.valueOf(pkE), BigInteger.valueOf(pkN)).longValue();
        return encoded;
    }
    public static int decodeWithPrivateKey(long N, int privateKey, int toDecode) {
        int decryptedCharacter = (int)BigInteger.valueOf(toDecode).modPow(BigInteger.valueOf(privateKey), 
                                                                          BigInteger.valueOf(N)).longValue();
        return decryptedCharacter;
    }
    public static int decodeWithPrivateKey(long pkP, long pkQ, int privateKey, int toDecode) {
        long N = pkP * pkQ;
        return decodeWithPrivateKey(N, privateKey, toDecode);
    }
}

Here we do some checks on the numbers

In [4]:
if (PrimeNumbers.isPrime(aliceP)) {
    System.out.printf("P (%d) is prime\n", aliceP);
} else {
    System.out.printf("P (%d) is not prime. Aborting.\n", aliceP);
    System.exit(1);
}
if (PrimeNumbers.isPrime(aliceQ)) {
    System.out.printf("Q (%d) is prime\n", aliceQ);
} else {
    System.out.printf("Q (%d) is not prime. Aborting.\n", aliceQ);
    System.exit(1);
}
System.out.printf("E is%s prime.\n", PrimeNumbers.isPrime(aliceE) ? "" : " not");

// We need D and ((p-1) x (q-1)) to be relative primes (aka coprimes)
boolean okCombo = RSAUtils.checkPQE(aliceP, aliceQ, aliceE);
System.out.printf("(E * D) mod \u03D5 = 1 ? %b\n", okCombo);
okCombo = RSAUtils.checkPhiDCoprime(aliceP, aliceQ, aliceE);
System.out.printf("\u03D5 & D coprimes ? %b\n", okCombo);

// For the fun:
long alicePxQ = aliceP * aliceQ; // aka N
System.out.printf("Alice P x Q (aka N) = %d: %s\n", 
                  alicePxQ, 
                  PrimeNumbers.spitMapOut((int)alicePxQ, 
                  PrimeNumbers.primeFactors((int)alicePxQ)));

long aliceN = alicePxQ;

P (127) is prime
Q (499) is prime
E is prime.
(E * D) mod ϕ = 1 ? true
ϕ & D coprimes ? true
Alice P x Q (aka N) = 63373: 63,373 = (499^1) x (127^1)


The two last methods of the class above are the keys of the process.  
Notice that both are using those Java `BigInteger`s, designed to deal with big numbers.  
Example:  
```
113 ^ 299 = 74208662054133427430283419284630423723086083250033980006048918082537216631039587269444325772963254321076798307775448236311678557274583841862495881031902103010966562227701461830001062437864235122845116992578539021526146447458066104183901983341715123850829323813213473167098640830428045766344873505056217088306739646383872867467058792684380293422334734506561634457828965683119038291079600349606340831612296147306892972691946313324059175658369615316424548699131445180263713472433919716056946934952847250285015852255572449823623047333743607992303288103494083536172706308350987589068481774411769548648147078635328244177
```
This number has 614 digits... Way beyond the capacity of a regular Java `Long`!

As seen above, we've done some checks:
- Are `P` and `Q` prime numbers?
  
We've defined a number named &phi; as `(P - 1) x (Q - 1)`  

- Is `(E * D) mod ϕ` equal to `1`?
- Are ϕ and D coprimes (is their PGCD equal to 1)?

And we've defined a number named `N`, as `P x Q`.

## For real, on one character
OK, it's give to give it a go.  
Let us **ENCODE** a character, like - let's say - `X`.  
Bernard wants to send Alice an `X`.  

### Encryption
As said before, Alice has published her **public key** into some phone book, where Bernard can find it. This key is the tuple `(E, N)`, in our present case, `(23_801, 127 x 499)`, which is `(23_801, 63_373)`.

In [5]:
int characterToEncrypt = (int)'X';
System.out.printf("Character to encrypt: %d\n", characterToEncrypt);
//
int encryptedWithPublicKey = RSAUtils.encodeWithPublicKey(aliceE, aliceN, characterToEncrypt);
System.out.printf("Encryption : %s (%c) => %s\n",
        NumberFormat.getInstance().format(characterToEncrypt),
        characterToEncrypt,
        NumberFormat.getInstance().format(encryptedWithPublicKey));

Character to encrypt: 88
Encryption : 88 (X) => 8,397


java.io.PrintStream@240df2dc

So, the `X` Bernard wants to send to Alice has been encrypted as `44021`. He sends this number to Alice.  

### Decryption
Alice receive this `8397`, she now needs to decrypt (or **DECODE**) it.

In [6]:
int privateKeyD = (int)RSAUtils.findD(aliceP, aliceQ, aliceE); // Private key
int decryptedWithPrivateKey = RSAUtils.decodeWithPrivateKey(aliceP, aliceQ, privateKeyD, encryptedWithPublicKey);
System.out.printf("Encrypted: %s, decrypted: %s (%c)\n",
        NumberFormat.getInstance().format(encryptedWithPublicKey),
        NumberFormat.getInstance().format(decryptedWithPrivateKey),
        decryptedWithPrivateKey);

Encrypted: 8,397, decrypted: 88 (X)


java.io.PrintStream@240df2dc

We're now back to the original message, `X`.  

### How it went
Let's at the two processes, more in details...  

#### The encryption process
if `x` is the character to encode, with `E` and `N`, both parts of the public key (with `N` = `P x Q`), the encoding is done by the formula
<!--```
enc = (x ^ E) mod N
```-->
$$
enc = (x^E) mod N
$$
Notice that:
- This is a one-way function
- As such, even if to know that you start from `x`, to finish with `enc`, even if you know the generic formula, finding the right values for `E` and `N` is going to be at least very difficult, and for sure very long.

#### The decryption process
This is where - to avoid the difficult and long process we just talked about - Alice is going to use her **private key**.  
This private key can be carefully stored by Alice, and it can also be recomputed, with the values of `P`, `Q`, and `E`.  
If we call this private key `D`, then `D` must satisfy the following:
$$
(E . D) mod ((P - 1) . (Q - 1)) = 1
$$
If, as before, we call &phi; the value of `((P - 1) x (Q - 1))`, we can write
$$
(E . D) mod \phi = 1
$$
And also, `D` and &phi; are coprime (their PGCD is equal to `1`).

In our very case, `D` is equal to `23`. Let's verify if it matches what we said.  
We have:
```
P = 127
Q = 499
E = 23,801
```

In [7]:
long P = 127;
long Q = 499;
long E = 23_801; // The public key

long D = 29; // The private key to verify

long PHI = ((P - 1) * (Q - 1));

long modulo = BigInteger.valueOf(E).multiply(BigInteger.valueOf(D)).mod(BigInteger.valueOf(PHI)).longValue();
System.out.printf("Modulo: %d, %s\n", modulo, (modulo == 1) ? "good!" : "Oops...");


long pgcdPhiD = PrimeNumbers.pgcd((int)D, (int)PHI);
System.out.printf("PGCD(D, \u03D5) = %d, %s\n", pgcdPhiD, (pgcdPhiD == 1) ? "good!" : "Oops...");

Modulo: 1, good!
PGCD(D, ϕ) = 1, good!


java.io.PrintStream@240df2dc

Those verifications are the ones performed by the methods `checkPQE` and `checkPhiDCoprime`, in the `RSAUtils` class.  
We can now tell that `23` is a valid value for our private key. It is certainly not the only one to fullfill those requirements!  
And this is also the reason to keep it secret.

To decode a character `enc`, with the private key (`E`, `N`), we apply the formula
$$
 x = (enc ^ E) mod N
$$


In [8]:
long enc = 8_397;
long N = P * Q;

long x = BigInteger.valueOf(enc).modPow(BigInteger.valueOf(D), BigInteger.valueOf(N)).longValue();
System.out.printf("Encrypted: %d, Decrypted: %d\n", enc, x);

Encrypted: 8397, Decrypted: 88


java.io.PrintStream@240df2dc

One last thing to notice here, is that you encode a `character` (8 bits), and end-up with an `int` or a `long`. 
The encoded message is going to take more space then the original one.

## For real, on a message
Basically, we are goig to do what we just did on one character, to a list of characters.  
We start from a `String`, and we will process - as previously - all the characters it contains.

In [9]:
String message = "Hello RSA World!";

We are going to get those characters (in a `byte[]`), and encode them, into a `List<Integer>`:

In [10]:
final byte[] bytes = message.getBytes();
List<Integer> encoded = new ArrayList<>(); // Warning !! Those are ints ! [0..&xFF]

System.out.println("-- Encoding --");
for (byte b : bytes) {
    int encodedByte = RSAUtils.encodeWithPublicKey(aliceE, N, b);
    System.out.printf("Encoding (%c) %d -> %s\n",
            b, b,
            NumberFormat.getInstance().format(encodedByte));
    encoded.add(encodedByte);
}

-- Encoding --
Encoding (H) 72 -> 3,426
Encoding (e) 101 -> 29,811
Encoding (l) 108 -> 27,833
Encoding (l) 108 -> 27,833
Encoding (o) 111 -> 51,927
Encoding ( ) 32 -> 12,351
Encoding (R) 82 -> 37,110
Encoding (S) 83 -> 11,232
Encoding (A) 65 -> 12,778
Encoding ( ) 32 -> 12,351
Encoding (W) 87 -> 58,839
Encoding (o) 111 -> 51,927
Encoding (r) 114 -> 14,334
Encoding (l) 108 -> 27,833
Encoding (d) 100 -> 61,209
Encoding (!) 33 -> 49,759


Now, to decode, we will first need the private key.  
We can get it from where we stored it, or re-calculate it, from our context.

In [11]:
int privateKeyD = (int)RSAUtils.findD(aliceP, aliceQ, aliceE); // Private key

The private key formula can have several solutions, let's see a few of them...

In [20]:
// Optional step.
boolean seePrivateKeyList = true; // Set to false at will...
if (seePrivateKeyList) {
    final List<Long> dList = RSAUtils.findDList(aliceP, aliceQ, aliceE); // Limited to Integer.MAX_VALUE !!
    System.out.printf("Possible Ds (private keys), %d entries (limited to 50 here...).\n", dList.size());
    dList.stream()
            .limit(50) // Limited to 50 items !!
            .forEach(k -> System.out.printf("\t-> %d\n", k));
    // Let's try to use the second one...
    privateKeyD = (int)dList.get(1).longValue();
    System.out.printf("- Will use private key %d\n", privateKeyD);
}

Possible Ds (private keys), 34224 entries (limited to 50 here...).
	-> 29
	-> 62777
	-> 125525
	-> 188273
	-> 251021
	-> 313769
	-> 376517
	-> 439265
	-> 502013
	-> 564761
	-> 627509
	-> 690257
	-> 753005
	-> 815753
	-> 878501
	-> 941249
	-> 1003997
	-> 1066745
	-> 1129493
	-> 1192241
	-> 1254989
	-> 1317737
	-> 1380485
	-> 1443233
	-> 1505981
	-> 1568729
	-> 1631477
	-> 1694225
	-> 1756973
	-> 1819721
	-> 1882469
	-> 1945217
	-> 2007965
	-> 2070713
	-> 2133461
	-> 2196209
	-> 2258957
	-> 2321705
	-> 2384453
	-> 2447201
	-> 2509949
	-> 2572697
	-> 2635445
	-> 2698193
	-> 2760941
	-> 2823689
	-> 2886437
	-> 2949185
	-> 3011933
	-> 3074681
- Will use private key 62777


Now, we can process to decryption

In [13]:
System.out.println("-- Decoding --");
List<Byte> decoded = new ArrayList<>();
encoded.stream().forEach(enc -> {
    byte decodedByte = (byte)RSAUtils.decodeWithPrivateKey(N, privateKeyD, enc); // N = P x Q
    System.out.printf("Decoded: %s => %d (%c)\n",
            NumberFormat.getInstance().format(enc),
            decodedByte,
            decodedByte);
    decoded.add(decodedByte);
});

-- Decoding --
Decoded: 3,426 => 72 (H)
Decoded: 29,811 => 101 (e)
Decoded: 27,833 => 108 (l)
Decoded: 27,833 => 108 (l)
Decoded: 51,927 => 111 (o)
Decoded: 12,351 => 32 ( )
Decoded: 37,110 => 82 (R)
Decoded: 11,232 => 83 (S)
Decoded: 12,778 => 65 (A)
Decoded: 12,351 => 32 ( )
Decoded: 58,839 => 87 (W)
Decoded: 51,927 => 111 (o)
Decoded: 14,334 => 114 (r)
Decoded: 27,833 => 108 (l)
Decoded: 61,209 => 100 (d)
Decoded: 49,759 => 33 (!)


Having done that, we can display the decrypted message, as a `String` produce from a `byte` array!

In [14]:
byte[] decodedBA = new byte[decoded.size()];
for (int i=0; i<decoded.size(); i++) {
    decodedBA[i] = decoded.get(i).byteValue();
}
System.out.printf("\nFinal message, decoded: [%s]\n", new String(decodedBA));


Final message, decoded: [Hello RSA World!]


java.io.PrintStream@240df2dc

## Dependencies
Here we explain (as we did before) how `P`, `Q`, `E`, and `D`, depend on each other.

We implement a class `ToolBox`, implementing a `findCandidateE` method, that might be able to suggest a valid public key for a given context.

In [15]:
class ToolBox {
    /**
     * Find a list of candidate public keys E
     * Careful... Might loop forever (findD)...
     * @param p
     * @param q
     * @param tentativeD
     * @return
     */
    public static List<Long> findCandidateE(long p, long q, long tentativeD) {
        List<Long> suggestions = new ArrayList<>();
        final int NB_KEYS = 10;

        // phi & d, coprime
        long phi = (p - 1) * (q - 1);

        long D = tentativeD; // the private key. Can be any int.

        long pgcdPhiD = PrimeNumbers.pgcd((int)D, (int)phi);
        if (pgcdPhiD != 1) {
            throw new RuntimeException(String.format("Bad combination, GCD = %d, \u03D5 => %s, D => %s", pgcdPhiD,
                    spitMapOut((int)phi, primeFactors((int)phi)), spitMapOut((int)D, primeFactors((int)D))));
        }
        // (E × D) mod Ø(n) = 1
        // Loop on E...
        long E = 0;
        boolean ok; // = ((E * D) % phi) == 1;
        while (suggestions.size() < NB_KEYS) {
            E += 1;
            ok = ((E * D) % phi) == 1;
            if (E == Long.MAX_VALUE) {
                System.out.printf("E reached value %d, Exiting.", Long.MAX_VALUE);
                break;
            }
            if (ok) {
                suggestions.add(E);
                // We need (E × D) mod Ø(n) = 1
                long modulo = (E * D) % phi;
                System.out.printf("\tFor E = %d, modulo = %d\n", E, modulo);

                long key = RSAUtils.findD(p, q, E); // Find Private Key D, with Public Key E
                System.out.printf("\t\tp=%d, q=%d, E=%d, make a D key: %d\n", p, q, E, key);
            }
        }
        System.out.printf("Exit the loop with E = %d, and size = %d\n", E, suggestions.size());

        return suggestions; // E;
    }
}

Let's see how it works in our previous context

In [16]:
long p = 127, q = 499, d = 29;
// long p = 127, q = 499, d = 126746; // To see an error
try {
    List<Long> testE = ToolBox.findCandidateE(p, q, d);
    System.out.println("-------");
    System.out.printf("For P: %d, Q: %d, D: %d\n", p, q, d);
    testE.stream().forEach(e -> {
        System.out.printf("Candidate E: %d (prime: %b)\n", e, PrimeNumbers.isPrime(e));
    });
} catch (Exception ex) {
    ex.printStackTrace();
}

jdk.jshell.spi.SPIResolutionException: resolution exception
	at REPL.$JShell$64$ToolBox.findCandidateE($JShell$64.java:19)
	at REPL.$JShell$68.do_it$($JShell$68.java:19)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:566)
	at io.github.spencerpark.ijava.execution.IJavaExecutionControl.lambda$execute$1(IJavaExecutionControl.java:95)
	at java.base/java.util.concurrent.FutureTask.run(FutureTask.java:264)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1128)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:628)
	at java.base/java.lang.Thread.run(Thread.java:834)


#### And now, let's try to crack it...  
When encoding the message, with the public key, we have `E`, and `N`.  
- `E` is 23801
- `N` is 63373

So, let's try to hack that stuff...    
Knowing that `N = P * Q`, and that `P` is a prime number, as well as `Q`, decomposing `N` in prime factors should give us `P` and `Q`:
<!--```
N 63373 => 63,373 = (499^1) x (127^1)
```-->
$$
 63,373 = (499^1) . (127^1)
$$

To decode the encrypted character, we need `N` (or `P` and `Q`), and the private key, as in the signature of `decodeWithPrivateKey(long pkP, long pkQ, int privateKey, int toDecode)`. The question that remains is "how can we get to the private key?".

The `RSAUtil.findD` method might help:

In [17]:
final long privateK = RSAUtils.findD(P, Q, E);
System.out.printf("With P %d, Q %d, E %d, found private key %d\n", P, Q, E, privateK);

With P 127, Q 499, E 23801, found private key 29


java.io.PrintStream@240df2dc

...Are we suppposed to find this that easily 🤔 ?

From the "Code Book", Annex J, we have
- E = 7
- N = 187

Those two `E` and `N` are the constituents of the public key, the one from the phone book anyone can access to encode a message for Alice.

To encode the character `X` (88), we use 
$$
enc = (88^7) mod (187) = 11
$$
Let's see if we can obtain `P` and `Q` from `N`:
$$
187 = (17^1) . (11^1)
$$
So `P` = 17, and `Q` = 11.

Just like before, let's try this:

In [18]:
P = 17;
Q = 11;
E = 7;
final long newD = RSAUtils.findD(P, Q, E);
System.out.printf("With P %d, Q %d, E %d, found private key %d\n", P, Q, E, newD);

With P 17, Q 11, E 7, found private key 23


java.io.PrintStream@240df2dc

In [19]:
int dec = RSAUtils.decodeWithPrivateKey(P * Q, (int)newD, 11);
System.out.printf("Decoded character: %d\n", dec);

Decoded character: 88


java.io.PrintStream@240df2dc

The private key we've just found is `23`. It happens to be the right one, and the decoded character happens to be `88`, which is right too.  
Mmmh... We just cracked it 🤔.  


See this [geeksforgeeks](https://www.geeksforgeeks.org/how-to-solve-rsa-algorithm-problems/) to test this...  
Also this [good paper](https://cybertalents.com/blog/rsa-encryption#:~:text=These%20are%20some%20real%2Dworld,Securing%20P2P%20data%20transfer.)
More soon here.

---
No next section (yet).
<!-- [Next Section](./Encryption.104.ipynb) -->